In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!unzip -q "SpeechAssign2_Q2/malayalam.zip" -d "/content"

replace /content/malayalam/test_known/audio/844424930309504-1145-m.m4a? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [3]:
!unzip -q "/content/drive/MyDrive/malayalam.zip" -d "/content"

replace /content/malayalam/test_data.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [4]:
!pip install s3prl

In [5]:
import torch
import torchaudio
from s3prl.nn import S3PRLUpstream
import os
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.metrics import roc_curve, auc
import soundfile as sf
from scipy.spatial.distance import cosine

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the Kathbath Dataset test set
test_audio_path = "/content/malayalam/test_known/audio"
audio_files = sorted(os.listdir(test_audio_path))

# Load reference and non-reference pairs from the text file
pairs_file = "/content/malayalam/test_known_data.txt"

/usr/local/lib/python3.10/dist-packages/s3prl/upstream/byol_s/byol_a/common.py:20: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("sox_io")


In [6]:
pairs_file = "/content/malayalam/test_known_data.txt"
lines = open(pairs_file).read().splitlines()

reference_pairs = []
non_reference_pairs = []

audio_file = set()

for line in lines:
    parts = line.strip().split(")
    if len(parts) < 3:
        # Skip this line if it doesn't have all three parts
        continue

    label = int(parts[0])
    # Transform the paths and remove the numeric directory
    ref_path_parts = parts[1].replace("/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e_data/pilot4_karya/aug_dc_data_raw/dc_data_clean_wav/malayalam/test_known/audio", "/content/malayalam/test_known/audio").split('/')
    ref_path_parts.remove(ref_path_parts[-2])  # Remove the numeric directory
    ref_path = '/'.join(ref_path_parts).replace(".wav", ".m4a")

    non_ref_path_parts = parts[2].replace("/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e_data/pilot4_karya/aug_dc_data_raw/dc_data_clean_wav/malayalam/test_known/audio", "/content/malayalam/test_known/audio").split('/')
    non_ref_path_parts.remove(non_ref_path_parts[-2])  # Remove the numeric directory
    non_ref_path = '/'.join(non_ref_path_parts).replace(".wav", ".m4a")

    if label == 1:
        reference_pairs.append((ref_path, non_ref_path))
    else:
        non_reference_pairs.append((ref_path, non_ref_path))
    audio_file.add(ref_path)
    audio_file.add(non_ref_path)


In [7]:
len(audio_file)

1767

In [8]:
len(lines)

50000

In [9]:
# Load the model
model = S3PRLUpstream("wavlm_large")
model = model.to(device)
model.eval()

with torch.no_grad():
    wavs = torch.randn(2, 16000 * 2).to(device)
    wavs_len = torch.LongTensor([16000 * 1, 16000 * 2]).to(device)
    all_hs, all_hs_len = model(wavs, wavs_len)

for hs, hs_len in zip(all_hs, all_hs_len):
    # assert isinstance(hs, torch.FloatTensor)
    # assert isinstance(hs_len, torch.LongTensor)

    batch_size, max_seq_len, hidden_size = hs.shape
    assert hs_len.dim() == 1

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [10]:
print(hs.shape)

torch.Size([2, 100, 1024])


In [11]:
audio_file_list = list(audio_file)

In [12]:
a = audio_file_list[0]
audio,_ = torchaudio.load(a)
# model.forward()

In [13]:
wavs.shape, wavs_len

(torch.Size([2, 32000]), tensor([16000, 32000], device='cuda:0'))

In [14]:
model.forward(wavs,wavs_len)[0][-1].shape

torch.Size([2, 100, 1024])

In [17]:
from tqdm import tqdm
diction = {}
# Function to extract features from audio files
def extract_features(model, audio_files):
    all_hs = []
    all_hs_len = []
    for audio_file in tqdm(audio_files, desc="Extracting features"):
        # Load audio file and convert to tensor
        audio, sample_rate = torchaudio.load(audio_file)
        audio = audio.to(device)  # Move audio to the device
        # Preprocess audio if needed (e.g., resampling, normalization)
        # Reshape audio tensor to match expected dimensions
        if audio.dim() == 1:  # If mono-channel audio
            audio = audio.unsqueeze(0)  # Add batch dimension
        # Compute features from the audio using the model
        with torch.no_grad():
            hs, _ = model(audio, torch.LongTensor([audio.size(1)]))
            # print(hs)
            hs = hs[-1].cpu().detach().numpy()
            # hs_len = hs_len.cpu()
            diction[audio_file] = hs
            # all_hs_len.append(hs_len)
            del audio
    # return all_hs


In [ ]:
# Extract features for reference pairs
extract_features(model, audio_file_list)


Extracting features:  98%|█████████▊| 1725/1767 [19:29<07:19, 10.45s/it]

In [ ]:
import os
import pickle

# Specify the directory path where you want to save the dictionary
directory_path = "/content/drive/MyDrive/SpeechAssign_Q1"
file_name = "dictionary_data.pkl"
file_path = os.path.join(directory_path, file_name)

# Serialize and save the dictionary to the file
with open(file_path, "wb") as file:
    pickle.dump(diction, file)

print("Dictionary saved successfully to:", file_path)


In [ ]:

# Extract features for non-reference pairs
non_reference_features, _ = extract_features(model, [non_ref_path for _, non_ref_path in non_reference_pairs])

In [ ]:
# Compute cosine similarity between reference and non-reference pairs
cosine_similarity_scores = []
for ref_feature, non_ref_feature in zip(reference_features, non_reference_features):
    cosine_similarity = compute_cosine_similarity(ref_feature, non_ref_feature)
    cosine_similarity_scores.extend(cosine_similarity)

# Create labels (1 for reference pairs, 0 for non-reference pairs)
labels = [1] * len(reference_features) + [0] * len(non_reference_features)

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(labels, cosine_similarity_scores)

# Compute EER
eer_threshold = thresholds[np.argmin(np.abs(fpr - (1 - tpr)))]
eer = fpr[np.argmin(np.abs(fpr - (1 - tpr)))]

# Step 2: Report the EER
print("Equal Error Rate (EER): {:.2f}%".format(eer * 100))